In [36]:
import visual_behavior_research.utilities as vbu
import visual_behavior_research.projects.tbd.database_utilities as du
import visual_behavior_research.projects.tbd.utilities as tbdu

import imageio

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import os
import sys
import json

import helper_functions
import internal_functions

In [3]:
data_path = os.path.join(os.path.split(os.getcwd())[0],'data')

In [4]:
os.listdir(data_path)

['session_list.csv',
 'README.md',
 'active_sessions',
 'histology_images',
 'anesthesia_sessions',
 'manually_rejected_cells.csv',
 '.gitignore',
 'passive_sessions']

In [38]:
data_summary = tbdu.get_db_table('summary')
mice_table = tbdu.get_db_table('mice')
data_summary = data_summary.drop(columns='id').merge(
    mice_table.drop(columns=['id','genotype']),
    left_on='mouse_id',
    right_on='mouse_id'
)

In [39]:
data_summary

,basepath,behavior_stage,folder,genotype,good_behavior,good_for_analysis,good_neural_data,good_sync,manually_checked,mouse_id,notes,problem,problem_description,processed,running_data,verified_files_present,size(gb),experiment_type,single_cell_plots_made,single_cell_images_made,number_accepted_cells,full_path,recording_date,processing_date,summarized,anterior_or_posterior,AP_coord,ML_coord,DV_coord,basefolder
0,//olsenlab1/data/endoscope_imaging,autoreward,2018.03.07_M332849_DoC_day1_autoreward,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,yes,yes,yes,yes,0,M332849,None,1,Not organized into 10 minute chunks,0.0,yes,None,7.479076e+01,change_detection,None,None,NaN,//olsenlab1/data/endoscope_imaging/M332849/201...,2018-03-07 15:16:45.876000,2018-12-15 15:53:28.984200001-08:00,None,anterior,1.5,3.125,2.50,//olsenlab1/data/endoscope_imaging/M332849
1,//olsenlab1/data/endoscope_imaging,static_grating,2018.03.09_M332849_DoC_day2,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,no,no,yes,yes,1,M332849,None,0,None,1.0,yes,None,1.517899e+02,change_detection,True,True,45.0,//olsenlab1/data/endoscope_imaging/M332849/201...,2018-03-09 11:27:50.579000,2018-07-25 12:41:22.762362242-07:00,True,anterior,1.5,3.125,2.50,//olsenlab1/data/endoscope_imaging/M332849
2,//olsenlab1/data/endoscope_imaging,static_grating,2018.03.13_M332849_DoC_day4,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,yes,yes,yes,yes,1,M332849,None,0,None,1.0,yes,None,1.341752e+02,change_detection,True,True,64.0,//olsenlab1/data/endoscope_imaging/M332849/201...,2018-03-13 11:26:45.003000,2018-07-20 10:57:53.647888899-07:00,True,anterior,1.5,3.125,2.50,//olsenlab1/data/endoscope_imaging/M332849
3,//olsenlab1/data/endoscope_imaging,natural_images,2018.03.30_M332849_DoC_day9,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,yes,yes,yes,yes,1,M332849,None,0,Note: more dead cells in forcus this day,1.0,yes,None,1.461645e+02,change_detection,True,True,57.0,//olsenlab1/data/endoscope_imaging/M332849/201...,2018-03-30 13:53:41.511000,2018-07-25 13:48:01.009045124-07:00,True,anterior,1.5,3.125,2.50,//olsenlab1/data/endoscope_imaging/M332849
4,//olsenlab1/data/endoscope_imaging,natural_images,2018.04.3_M332849_DoC_day11,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,no,no,yes,no,1,M332849,None,0,None,1.0,yes,None,1.443961e+02,change_detection,True,True,108.0,//olsenlab1/data/endoscope_imaging/M332849/201...,2018-04-03 13:20:29.101000,2018-07-25 13:59:23.514958143-07:00,True,anterior,1.5,3.125,2.50,//olsenlab1/data/endoscope_imaging/M332849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,//olsenlab1/data/endoscope_imaging,None,2019.06.05_M433602_EEG_EMG_iso,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,None,None,None,None,None,M433602,None,1,local variable 'rec_files' referenced before a...,NaN,None,None,1.413748e-06,None,None,None,NaN,//olsenlab1/data/endoscope_imaging/M433602/201...,0,None,None,anterior,1.0,3.125,2.00,//olsenlab1/data/endoscope_imaging/M433602
265,//olsenlab1/data/endoscope_imaging,None,2019.06.04_iso,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,None,None,None,None,None,M435529,None,1,local variable 'rec_files' referenced before a...,NaN,None,None,7.906929e-07,None,None,None,NaN,//olsenlab1/data/endoscope_imaging/M435529/201...,0,None,None,posterior,0.0,4.000,2.75,//olsenlab1/data/endoscope_imaging/M435529
266,//olsenlab1/data/endoscope_imaging,None,2019.06.10_EEG_EMG_test_newconfig,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,None,None,None,None,None,M435529,None,1,local variable 'rec_files' referenced before a...,NaN,None,None,6.913345e-01,None,None,None,NaN,//olsenlab1/data/endoscope_imaging/M435529/201...,0,None,None,posterior,0.0,4.000,2.75,//olsenlab1/data/endoscope_imaging/M435529
267,//olsenlab1/data/endoscope_imaging,None,2019.06.10_EEG_EMG_test_oldconfig,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,None,None,None,None,None,M435529,None,1,local variabl

## get anesthesia sessions to analyze

In [49]:
anesthesia_sessions_to_analyze = data_summary[
    (data_summary['experiment_type']=='iso_combined')
    &(data_summary['good_for_analysis']!='no')
    &(data_summary['anterior_or_posterior']=='anterior')
].copy().reset_index()
anesthesia_sessions_to_analyze['session_type'] = 'anesthesia'
anesthesia_sessions_to_analyze

,index,basepath,behavior_stage,folder,genotype,good_behavior,good_for_analysis,good_neural_data,good_sync,manually_checked,mouse_id,notes,problem,problem_description,processed,running_data,verified_files_present,size(gb),experiment_type,single_cell_plots_made,single_cell_images_made,number_accepted_cells,full_path,recording_date,processing_date,summarized,anterior_or_posterior,AP_coord,ML_coord,DV_coord,basefolder,session_type
0,34,//olsenlab1/data/endoscope_imaging,None,2018.02.27_M334412_combined,Gnb4-IRES2-Cre;AAV-GCaMP6s,None,None,None,None,None,M334412,None,None,None,1.0,None,None,42.398704,iso_combined,True,True,148.0,//olsenlab1/data/endoscope_imaging/M334412/201...,0,2019-03-14 17:37:57.629808903-07:00,True,anterior,1.5,2.625,2.7,//olsenlab1/data/endoscope_imaging/M334412,anesthesia
1,57,//olsenlab1/data/endoscope_imaging,None,2018.02.20_M348126_combined,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai94(TITL-...,None,None,None,None,None,M348126,None,None,None,1.0,None,None,34.248248,iso_combined,True,True,85.0,//olsenlab1/data/endoscope_imaging/M348126/201...,0,2019-03-14 19:28:13.741096020-07:00,True,anterior,1.5,2.625,2.5,//olsenlab1/data/endoscope_imaging/M348126,anesthesia
2,77,//olsenlab1/data/endoscope_imaging,None,2018.02.29_M339255_combined,Gnb4-IRES2-Cre;AAV-GCaMP6s,None,None,None,None,None,M339255,None,None,None,1.0,None,None,45.705585,iso_combined,True,True,125.0,//olsenlab1/data/endoscope_imaging/M339255/201...,0,2019-03-14 23:05:12.098660707-07:00,True,anterior,1.5,2.625,2.7,//olsenlab1/data/endoscope_imaging/M339255,anesthesia
3,106,//olsenlab1/data/endoscope_imaging,None,2017.11.17_M334410_combined,Gnb4-IRES2-Cre;AAV-GCaMP6s,None,None,None,None,None,M334410,None,None,None,1.0,None,None,39.177161,iso_combined,True,True,103.0,//olsenlab1/data/endoscope_imaging/M334410/201...,0,2019-03-15 02:11:08.873144865-07:00,True,anterior,1.5,2.625,2.5,//olsenlab1/data/endoscope_imaging/M334410,anesthesia
4,135,//olsenlab1/data/endoscope_imaging,None,2017.11.29_M335379_combined,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai94(TITL-...,None,None,None,None,None,M335379,None,None,None,1.0,None,None,41.983551,iso_combined,True,True,119.0,//olsenlab1/data/endoscope_imaging/M335379/201...,0,2019-03-15 05:08:14.018605709-07:00,True,anterior,1.5,3.125,2.5,//olsenlab1/data/endoscope_imaging/M335379,anesthesia
5,156,//olsenlab1/data/endoscope_imaging,None,20180921_M375765_combined,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,None,None,None,None,None,M375765,None,None,None,1.0,None,None,35.151025,iso_combined,True,True,21.0,//olsenlab1/data/endoscope_imaging/M375765/201...,0,2019-03-15 07:12:19.855928898-07:00,True,anterior,1.5,2.625,2.5,//olsenlab1/data/endoscope_imaging/M375765,anesthesia
6,169,//olsenlab1/data/endoscope_imaging,None,20180921_M397408_combined,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,None,None,None,None,None,M397408,None,None,None,1.0,None,None,31.976700,iso_combined,True,True,35.0,//olsenlab1/data/endoscope_imaging/M397408/201...,0,2019-03-13 20:32:17.976514578-07:00,True,anterior,1.5,2.625,2.5,//olsenlab1/data/endoscope_imaging/M397408,anesthesia
7,196,//olsenlab1/data/endoscope_imaging,None,2019.03.22_combined,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,None,None,None,None,None,M416531,None,None,None,1.0,None,None,32.533512,iso_combined,True,True,64.0,//olsenlab1/data/endoscope_imaging/M416531/201...,0,2019-03-25 16:23:02.121844530-07:00,True,anterior,1.5,3.125,2.5,//olsenlab1/data/endoscope_imaging/M416531,anesthesia


## get passive sessions to analyze

In [11]:
all_passive = data_summary[
    (data_summary['experiment_type'] == 'passive_stim')
    &(data_summary['good_for_analysis']!='False')
#     &(summary['anterior_or_posterior']=='anterior')
].copy().sort_values(by=['mouse_id','experiment_type'])

# merge in lens location data from mouse table 
all_passive = all_passive.merge(mice_table[['mouse_id','anterior_or_posterior', 'AP_coord', 'ML_coord', 'DV_coord']],on='mouse_id').sort_values(by='recording_date')

# keep only anterior sessions:
all_passive = all_passive[
    (all_passive['anterior_or_posterior']=='anterior')
]

# keep only last experiment for each mouse (in some cases we repeated passive stim experiments)
all_passive = all_passive[all_passive[['mouse_id']].duplicated(keep='last')==False].reset_index()

# add a genotype shorthand column for display
all_passive['genotype_shorthand'] = all_passive['genotype'].map(lambda x: internal_functions.get_genotype_shorthand(x))

# sort and reset indices
passive_sessions_to_analyze = all_passive.sort_values(by=['genotype','anterior_or_posterior']).reset_index()

passive_sessions_to_analyze['sex'] = passive_sessions_to_analyze['mouse_id'].map(
    lambda mid: internal_functions.get_gender(mid.lstrip('M'))
)
cols_to_display = [
    'genotype_shorthand',
    'genotype',
    'mouse_id',
    'AP_coord', 
    'ML_coord', 
    'DV_coord',
    'sex', 
    'number_accepted_cells',
    'full_path'
]

passive_sessions_to_analyze['session_type'] = 'passive'

passive_sessions_to_analyze[cols_to_display]

,genotype_shorthand,genotype,mouse_id,AP_coord,ML_coord,DV_coord,sex,number_accepted_cells,full_path
0,Gnb4-Cre;AAB-GCaMPs,Gnb4-IRES2-Cre;AAV-GCaMP6s,M334410,1.5,2.625,2.5,M,72.0,//olsenlab1/data/endoscope_imaging/M334410/201...
1,Gnb4-Cre;AAB-GCaMPs,Gnb4-IRES2-Cre;AAV-GCaMP6s,M334412,1.5,2.625,2.7,M,146.0,//olsenlab1/data/endoscope_imaging/M334412/201...
2,Gnb4-CreERT2;Ai148,Gnb4-IRES2-CreERT2/wt;Ai148(TIT2L-GC6f-ICL-tTA...,M406494,1.5,2.625,2.5,F,20.0,//olsenlab1/data/endoscope_imaging/M406494/201...
3,Gnb4-CreERT2;Ai93,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,M332849,1.5,3.125,2.5,M,11.0,//olsenlab1/data/endoscope_imaging/M332849/201...
4,Gnb4-CreERT2;Ai93,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,M397408,1.5,2.625,2.5,F,17.0,//olsenlab1/data/endoscope_imaging/M397408/201...
5,Gnb4-CreERT2;Ai93,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,M375765,1.5,2.625,2.5,M,10.0,//olsenlab1/data/endoscope_imaging/M375765/201...
6,Gnb4-CreERT2;Ai93,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,M416531,1.5,3.125,2.5,F,61.0,//olsenlab1/data/endoscope_imaging/M416531/201...
7,Gnb4-CreERT2;Ai93,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,M432805,1.0,3.125,2.0,M,45.0,//olsenlab1/data/endoscope_imaging/M432805/201...
8,Gnb4-CreERT2;Ai94,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai94(TITL-...,M335379,1.5,3.125,2.5,M,130.0,//olsenlab1/data/endoscope_imaging/M335379/201...
9,Gnb4-CreERT2;Ai94,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai94(TITL-...,M351181,1.5,2.625,2.5,F,118.0,//olsenlab1/data/endoscope_imaging/M351181/201...


## get active sessions to analyze
use last session for each of the 4 mice

In [12]:
last_sessions = pd.DataFrame(data_summary.query('good_for_analysis == "yes"').groupby('mouse_id')['recording_date'].max()).reset_index() # last session for every mouse
active_sessions_to_analyze = last_sessions.merge(
    data_summary,
    left_on=['mouse_id','recording_date'],
    right_on=['mouse_id','recording_date'],
)
active_sessions_to_analyze['session_type'] = 'active'

active_sessions_to_analyze['sex'] = active_sessions_to_analyze['mouse_id'].map(
    lambda mid: internal_functions.get_gender(mid.lstrip('M'))
)
active_sessions_to_analyze = active_sessions_to_analyze.merge(
    mice_table[['mouse_id','anterior_or_posterior', 'AP_coord', 'ML_coord', 'DV_coord']],
    on='mouse_id'
)
active_sessions_to_analyze = active_sessions_to_analyze.query('anterior_or_posterior == "anterior"')

active_sessions_to_analyze

,mouse_id,recording_date,id,basepath,behavior_stage,folder,genotype,good_behavior,good_for_analysis,good_neural_data,...,number_accepted_cells,full_path,processing_date,summarized,session_type,sex,anterior_or_posterior,AP_coord,ML_coord,DV_coord
0,M332849,2018-06-14 12:44:33.658000,12,//olsenlab1/data/endoscope_imaging,natural_images,2018.06.14_M332849_DoC_day23,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,yes,yes,yes,...,126.0,//olsenlab1/data/endoscope_imaging/M332849/201...,2018-07-26 12:11:01.983524799-07:00,True,active,M,anterior,1.5,3.125,2.5
1,M348126,2018-06-13 13:47:38.463000,29,//olsenlab1/data/endoscope_imaging,natural_images,2018.06.13_M348126_DoC_Day20,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai94(TITL-...,yes,yes,yes,...,103.0,//olsenlab1/data/endoscope_imaging/M348126/201...,2018-07-26 18:15:45.828682184-07:00,True,active,F,anterior,1.5,2.625,2.5
2,M351181,2018-03-14 14:32:15.328000,37,//olsenlab1/data/endoscope_imaging,natural_images,2018.03.14__M351181_DoC_day8,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai94(TITL-...,yes,yes,yes,...,127.0,//olsenlab1/data/endoscope_imaging/M351181/201...,2018-07-26 18:59:36.293976068-07:00,True,active,F,anterior,1.5,2.625,2.5
3,M397408,2018-11-01 11:40:14.487000,153,//olsenlab1/data/endoscope_imaging,None,20181101_M397408_DoC_day5,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,None,yes,None,...,29.0,//olsenlab1/data/endoscope_imaging/M397408/201...,2019-01-22 01:32:10.700291872-08:00,True,active,F,anterior,1.5,2.625,2.5


In [50]:
cols_to_save = [
    'mouse_id',
    'sex',
    'genotype',
    'AP_coord', 
    'ML_coord', 
    'DV_coord',
    'session_type',
    'recording_date',
    'processing_date',
]
session_list = pd.concat((
    anesthesia_sessions_to_analyze,
    passive_sessions_to_analyze,
    active_sessions_to_analyze
))

session_list[cols_to_save].to_csv(
    os.path.join(data_path,'session_list.csv'),
    index=False
)

In [51]:
session_list[['session_type']]

,session_type
0,anesthesia
1,anesthesia
2,anesthesia
3,anesthesia
4,anesthesia
5,anesthesia
6,anesthesia
7,anesthesia
0,passive
1,passive


## load sessions from disk
This gets sessions from the internal file system

In [52]:
sessions = {
    'anesthesia':{},
    'active':{},
    'passive':{}
}
for idx,row in session_list.iterrows():
    if row['session_type'] == 'anesthesia':
        sessions[row['session_type']][row['mouse_id']] = internal_functions.load_anesthesia_session_from_disk(row['full_path'])
    elif row['session_type'] == 'passive':
        sessions[row['session_type']][row['mouse_id']] = internal_functions.load_passive_session_from_disk(row['full_path'])
    elif row['session_type'] == 'active':
        sessions[row['session_type']][row['mouse_id']] = internal_functions.load_active_session_from_disk(row['full_path'])
        
    print('on session {} of {}'.format(idx+1,len(session_list)),end='\r')

filename = //olsenlab1/data/endoscope_imaging/M332849/2018.06.14_M332849_DoC_day23/additional_data/180614134458-task=DoC_NaturalImages_CAMMatched_n=8_stage=natural_images_drop_reward_mouse=M332849.pkl
filename = //olsenlab1/data/endoscope_imaging/M348126/2018.06.13_M348126_DoC_Day20/additional_data/180613144805-task=DoC_NaturalImages_CAMMatched_n=8_stage=natural_images_drop_reward_mouse=M348126.pkl
filename = //olsenlab1/data/endoscope_imaging/M351181/2018.03.14__M351181_DoC_day8/additional_data/180314153237-task=DoC_NaturalImages_CAMMatched_n=8_stage=natural_images_drop_reward_mouse=M351181.pkl
filename = //olsenlab1/data/endoscope_imaging/M397408/20181101_M397408_DoC_day5/additional_data/181101124039-task=DoC_NaturalImages_CAMMatched_n=8_stage=natural_images_mouse=M397408.pkl


## get cells to keep after manual and automatic filtering for each session

In [22]:
manually_rejected_cells = pd.read_csv(os.path.join(data_path, 'manually_rejected_cells.csv'))

for session_type in ['passive','active']:
    mice = sessions[session_type].keys()
    for mouse in mice:
        # get metrics dataframe
        metrics = sessions[session_type][mouse].metrics
        # apply filters, get names of cells to keep
        cells_to_keep = metrics[
            (metrics['snr']>3)
            &(metrics['likely_repeat']==False) # exclude repeats
        ].reset_index()['cellName']

        # remove any cells that are in the manually rejected table
        for cell_id in cells_to_keep.values:
            rejected_cell_query = manually_rejected_cells.query('mouse_id == "{}" and session_type == "{}" and cell_id == "{}"'.format(
                mouse,
                session_type,
                cell_id
            ))
            if len(rejected_cell_query) > 0:
                cells_to_keep.drop(idx)

        # save the list of cells to keep as an attribute of the session object
        sessions[session_type][mouse].filtered_cell_IDs = cells_to_keep.values

In [23]:
manually_rejected_cells

,mouse_id,session_type,cell_id
0,M397408,active,C066
1,M397408,active,C020
2,M397408,active,C097
3,M397408,active,C042
4,M397408,active,C035
5,M397408,active,C055


## save session attributes as individual files
To be reconstructed into a single object for later analysis

In [27]:
force_overwrite = False
for session_type in ['anesthesia','passive','active']:
    session_type_folder = os.path.join(data_path,'{}_sessions'.format(session_type))
    helper_functions.make_folder(session_type_folder)
    for mouse in sessions[session_type].keys():
        mouse_folder = os.path.join(session_type_folder, mouse)
        helper_functions.make_folder(mouse_folder)

        # save trials (extended_trials for active sessions)
        if session_type == 'active':
            trials_key = 'extended_trials'
        elif session_type == 'passive':
            trials_key = 'trials'
        else:
            trials_key = None
        filename = os.path.join(mouse_folder, 'trials.csv')
        if not os.path.exists(filename) and not force_overwrite and trials_key:
            sessions[session_type][mouse].behavior_core_data[trials_key].drop(columns=['index']).to_csv(
                filename, 
                index=False
            )
        
        # save licks, rewards, visual_stimuli for active sessions:
        if session_type == 'active':
            for table_name in ['licks', 'rewards', 'visual_stimuli']:
                filename = os.path.join(mouse_folder, '{}.csv'.format(table_name))
                if not os.path.exists(filename) and not force_overwrite:
                    sessions[session_type][mouse].behavior_core_data[table_name].to_csv(
                        filename, 
                        index=False
                    )
                    
        # save event_dict for active sessions
        if session_type == 'active':
            filename = os.path.join(mouse_folder, 'events.json'.format(table_name))
            if not os.path.exists(filename) and not force_overwrite:
                helper_functions.save_json(
                    sessions[session_type][mouse].event_dict,
                    filename
                )

        # save_filtered_cell_ids:
        filename = os.path.join(mouse_folder, 'filtered_cell_ids.csv')
        if not os.path.exists(filename) and not force_overwrite:
            pd.DataFrame(sessions[session_type][mouse].filtered_cell_IDs).rename(columns={0:'cell_id'}).to_csv(
                filename, 
                index=False
            )

        # save traces
        filename = os.path.join(mouse_folder, 'traces.csv')
        if not os.path.exists(filename) and not force_overwrite:
            sessions[session_type][mouse].traces.to_csv(
                filename, 
                index=False
            )

        # save filtered_traces
        filename = os.path.join(mouse_folder, 'filtered_traces.csv')
        if not os.path.exists(filename) and not force_overwrite:
            sessions[session_type][mouse].filtered_traces.to_csv(
                filename, 
                index=False
            )

        # save metrics
        filename = os.path.join(mouse_folder, 'metrics.csv')
        if not os.path.exists(filename) and not force_overwrite:
            sessions[session_type][mouse].metrics.reset_index().to_csv(
                filename, 
                index=False
            )
            
        # save cell images
        cell_folder = os.path.join(mouse_folder, 'cell_images')
        helper_functions.make_folder(cell_folder)
        internal_functions.save_cell_images(
            sessions[session_type][mouse],
            cell_folder,
            force_overwrite,
        ) 